## Indicator Based Evolutionary Algorithm (IBEA) applied to neural models

In [1]:
%matplotlib inline
from __future__ import division
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

import likelihoodfree.io as io
import os

import bluepyopt as bpopt
from bluepyopt.parameters import Parameter

from lfmods.hh import HHSimulator

## Some parameters of the analysis

In [2]:
###############################################################################
# seed of likelihood-free run from which we extract pilot means, stds, observed data, ground-truth parameters
seed_lfree = 1

###############################################################################
# number of simulations per round
sim_step = 500
num_round = 5

## Load observed data, pilot means and pilot stds

In [3]:
# loading filenames, paths and data
prefix = 'hh_'+str(sim_step)+'_iwloss_svi_inc_data_seed'+str(seed_lfree)
basepath = '../results/hh/'
dir_nets = basepath + 'nets/'
dir_plots = basepath + 'plots/'
dir_ibea = basepath + 'ibea/'

# create folder for plots
if not os.path.exists(dir_ibea):
    os.makedirs(dir_ibea)

###############################################################################
# loading data and posteriors
dists, infos, losses, nets, posteriors, sims = io.load_prefix(dir_nets, prefix)

# data
l_sims = list(sims.items())
obs_stats = l_sims[len(sims)-1][1].obs
# y_obs = l_sims[len(sims)-1][1].obs_trace.reshape(-1,1)
labels_sum_stats = l_sims[len(sims)-1][1].labels_sum_stats

###############################################################################
# ground-truth parameters
gt = l_sims[0][1].true_params
n_params = l_sims[0][1].n_params

labels_params = l_sims[len(sims)-1][1].labels_params

###############################################################################
# time
t = l_sims[0][1].t
dt = l_sims[0][1].dt

###############################################################################
# input
I = l_sims[0][1].I_obs

###############################################################################
# pilot means and standard deviations
pilot_means = l_sims[len(sims)-1][1].pilot_means
pilot_stds = l_sims[len(sims)-1][1].pilot_stds

## Set up the HH simulator

In [4]:
sim = HHSimulator(seed=100+seed_lfree,pilot_samples=0.,cached_sims=False, cached_pilot=False)

###############################################################################
# un-logged prior over parameters
unlog_prior_min = sim.param_invtransform(l_sims[len(sims)-1][1].prior_min)
unlog_prior_max = sim.param_invtransform(l_sims[len(sims)-1][1].prior_max)

params1 = []
for i in range(n_params):
    params1.append(Parameter(labels_params[i], bounds=[unlog_prior_min[i], unlog_prior_max[i]]))

## Training algorithm

In [5]:
class hh_evaluator(bpopt.evaluators.Evaluator):
    def __init__(self, I, dt, t, target_stats, labels_sum_stats, pilot_means, pilot_stds, params1):
        self.I = I
        self.dt = dt
        self.t = t
        self.target_stats = target_stats
        self.pilot_means = pilot_means
        self.pilot_stds = pilot_stds

        super(hh_evaluator, self).__init__(objectives=labels_sum_stats,params=params1)


    def evaluate_with_lists(self, param_list):
        A = param_list

        # simulation
        params = np.array(A)
        hh = sim.bm.HH(-70.,params.reshape(1,-1))
        states = hh.sim_time(self.dt, self.t, self.I).reshape(1, -1, 1)

        # summary statistics
        sum_stats = sim.calc_summary_stats(states)
        sum_stats -= self.pilot_means
        sum_stats /= self.pilot_stds

        return np.ndarray.tolist(np.abs(sum_stats - self.target_stats)[0])



def run_deap(I, dt, t, params1, obs_stats, labels_sum_stats,
             pilot_means, pilot_stds, optim_ibea=1, offspring_size=10, max_ngen=10):

    # choose and run genetic algorithm
    evaluator = hh_evaluator(I, dt, t, obs_stats, labels_sum_stats, pilot_means, pilot_stds, params1)

    # choose genetic algorithm
    if optim_ibea==1:
        opt = bpopt.deapext.optimisations.IBEADEAPOptimisation(evaluator,offspring_size=offspring_size)
    else:
        opt = bpopt.deapext.optimisations.DEAPOptimisation(evaluator,offspring_size=offspring_size)

    final_pop, halloffame, log, hist = opt.run(max_ngen=max_ngen)

    io.save((final_pop,halloffame,log,hist), dir_ibea + 'ibea_offsp'
            + str(offspring_size) + '_ngen' + str(max_ngen) + '.pkl')

## Run IBEA algorithm

In [16]:
optim_ibea = 1
offspring_size = int(sim_step)
max_ngen = int(num_round)
# offspring_size = 3
# max_ngen = 3
t_sim = time.time()
run_deap(I, dt, t, params1, obs_stats, labels_sum_stats,
         pilot_means, pilot_stds, optim_ibea, offspring_size, max_ngen)

## Inspect output

In [17]:
# load deap results
_, halloffame, log,_ = io.load(dir_ibea + 'ibea_offsp'
                               + str(offspring_size) +'_ngen' + str(max_ngen) + '.pkl')
best_params = halloffame[0]
best_sumstats = best_params.fitness.values
hlf = np.asarray(best_params)
    
# print results
print(best_sumstats)
print(hlf)
print(gt)

(0.0, 0.023642502746821503, 0.02562500596275112, 0.020653044358985063, 0.014464677514425699, 0.008183946199190592, 0.002030670061806711, 0.0038279154984491026, 0.008657447107189353, 0.011703828003807204, 0.012367524516685602, 0.2973927804910952, 0.3623064101276262, 0.004784854483563028, 0.036148125482770466, 0.017416149546900073, 0.009553943427358158, 0.0015015486473244355, 0.0038320378545588785, 0.0069012562599987115)
[  4.45008794e+01   3.43365909e+00   6.18766909e-02   5.47420327e+01
   1.15634822e+02   6.18844164e+01   6.03176145e-02   7.91550547e+02
   7.12001941e-01   4.02249431e+01   5.55835800e+01   3.32404261e-01]
[  5.00000000e+01   5.00000000e+00   1.00000000e-01   5.00000000e+01
   9.00000000e+01   7.00000000e+01   7.00000000e-02   6.00000000e+02
   5.00000000e-01   4.00000000e+01   6.00000000e+01   5.00000000e-01]
